In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import time

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from sklearn.model_selection import train_test_split
raw_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Simulator/simulator_data.csv',
                    index_col = None)

train_y = raw_df['L_THRP_DL_BITRATE_LS5M_RATIO'].values

X_cols = raw_df.columns[1:-1]

train_X = raw_df[X_cols].values

#train_X,valid_X,train_y,valid_y = train_test_split(raw_X,raw_y,test_size=0.5,random_state = 2333)

train_X = np.expand_dims(train_X,-1)

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(256,activation = 'relu',input_dim = 156,kernel_initializer = 'normal'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(128,activation = 'relu',kernel_initializer = 'normal'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(64,activation = 'relu',kernel_initializer = 'normal'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1,activation = 'linear',bias_initializer = 'zero'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               40192     
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total para

In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=train_X.shape[1:]))
model.add(tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=train_X.shape[1:]))
model.add(tf.keras.layers.Conv1D(128, 5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size= 2))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=train_X.shape[1:]))
model.add(tf.keras.layers.Conv1D(256, 5, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2, strides=2,))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
#model.add(tf.keras.layers.Activation('softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1 (Batc (None, 156, 1)            4         
_________________________________________________________________
conv1d (Conv1D)              (None, 156, 64)           384       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 78, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 78, 64)            0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 78, 64)            256       
_________________________________________________________________
conv1d_1 (Conv1D)    

In [25]:
#from tf.keras.models import load_model

#model = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Simulator/first_model.h5')

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=tf.keras.losses.mse,
    metrics = ['mae']
)



# with a Sequential model
#get_3rd_layer_output = K.function([model.layers[0].input],
#                                  [model.layers[3].output])
#layer_output = get_3rd_layer_output([train_X[:10]])

#print(layer_output)

history = model.fit(train_X, train_y, epochs=30, 
                    batch_size=32,  verbose=1, validation_split=0.4,)

model.save('/content/gdrive/My Drive/Colab Notebooks/Simulator/first_model.h5')


Train on 116602 samples, validate on 77735 samples
Epoch 1/30
116602/116602 [==============================] - 68s 580us/sample - loss: 37.8521 - mean_absolute_error: 3.6769 - val_loss: 39.7738 - val_mean_absolute_error: 3.4924
Epoch 2/30
 89728/116602 [======================>.......] - ETA: 13s - loss: 36.6921 - mean_absolute_error: 3.6354

KeyboardInterrupt: ignored

In [9]:
#from tf.keras.models import load_model
from tensorflow.keras import backend as K

model = tf.keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/Simulator/first_model.h5')

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[16].output])
layer_output = get_3rd_layer_output([train_X[:50]])

print(layer_output[0] - np.reshape(train_y[:50],(50,1)))

[[ 0.09799934]
 [ 3.30694371]
 [ 4.37334161]
 [-0.16374893]
 [-0.7594193 ]
 [ 2.78321581]
 [-2.85770305]
 [-3.83740966]
 [ 1.50517876]
 [ 1.54421911]
 [ 1.81115006]
 [-0.35216341]
 [ 1.31799201]
 [-1.62606581]
 [-0.40406803]
 [ 1.73437888]
 [ 1.45745118]
 [ 2.41717943]
 [ 2.93119261]
 [-1.18072747]
 [-5.09566666]
 [-4.10698958]
 [ 2.91981506]
 [-0.92373616]
 [ 1.97360659]
 [ 2.99541705]
 [ 0.70862138]
 [ 1.27052784]
 [ 0.43887955]
 [ 1.51010942]
 [ 6.63183094]
 [ 1.14612323]
 [-3.68823836]
 [-0.58433951]
 [-1.01555073]
 [ 0.81967572]
 [-1.92375752]
 [ 3.67772652]
 [ 3.07884473]
 [-2.1994548 ]
 [-1.21474475]
 [-4.64210186]
 [ 2.62608539]
 [ 2.83145735]
 [-0.01839032]
 [-5.29854144]
 [ 1.63222555]
 [ 3.4158172 ]
 [ 1.68597552]
 [ 0.84528012]]


In [7]:
import os
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)
tpu_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3),
    loss=tf.keras.losses.mse,
    metrics=['mae']
)

def train_gen(batch_size):
  while True:
    offset = np.random.randint(0, train_X.shape[0] - batch_size)
    yield train_X[offset:offset+batch_size], train_y[offset:offset + batch_size]
    

tpu_model.fit_generator(
    train_gen(32),
    epochs=15,
    steps_per_epoch = 1000,
)


tpu_model.save('/content/gdrive/My Drive/Colab Notebooks/Simulator/first_model.h5')

INFO:tensorflow:Querying Tensorflow master (grpc://10.31.148.114:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4431540867002430331)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17166999476251181485)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15000605796292243911)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17953433589016830482)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10960019856741254936)
INFO:tensorflow:*** Available Device: _DeviceAttribut